In [ ]:
import pandas as pd
import numpy as np


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

import string
import re

import spacy
import lxml

In [ ]:
BeautifulSoup()

In [ ]:
# driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe')

# jobs = ['data-scientist']
# # , 'data-analyst', 'business-analyst','research-scientist','machine-learning-engineer'
# raw = []
# for job in jobs:
    
#     url = 'https://www.seek.com.au/'+job+'-jobs/'

#     driver.get(url)
#     WebDriverWait(driver, 3)

#     pages = 3
    
    
#     for page in range(2,pages):
#         ads = driver.find_elements_by_css_selector('._2iNL7wI')
#         links = [ad.get_attribute('href') for ad in ads]
#         #print(ads)
#         for link in links[2:]:
#             driver.get(link)
#             WebDriverWait(driver, 3)
#             raw_html = driver.page_source
#             raw.append([raw_html])
#             driver.back()
#             WebDriverWait(driver, 3)
#         driver.get(url+'?page='+str(page))
# raw_df = pd.Series(raw)

# driver.close()

# raw_df.to_csv('./raw_data.csv',index=False)

for i in raw_df:
    
    ad_list = []
    
    salarys = []
    
    soup = BeautifulSoup(i[0],'lxml').text.strip()
#     print(type(soup))
    salary = re.findall('.*\$(\d+).*',soup)
    #print(soup.text)
#     .findall('.*\$(\d+).*')
    #break
    
    #print(salary)
    if salary:
        
        ad_list.append(soup)
        
        day = re.compile('.*\d+\bday')
        pa = re.compile('.*\d+\bp\.a\.|.*\d+\bp\s?a')
        hour = re.compile('.*\d+\bp[\.\s]h.')
    
        if re.search(day,soup):
            
            salary = [float(sal)*5*48 for sal in salary if float(sal)>0]
            print(salary,str(1))
            salarys.append(np.median(salary))
            
    
        elif re.search(pa,soup):
            
            salary = [float(sal)*1000 for sal in salary if float(sal)>0]
            print(salary,str(2))
            salarys.append(np.median(salary))

        elif re.search(hour,soup):
            
            salary = [float(sal)*24*5*48 for sal in  salary if float(sal)>0]
            print(salary,str(3))
            salarys.append(np.median(salary))
        
        else:
            
            salary = [float(sal) for sal in  salary if float(sal)>0]
            print(salary,str(4))
            salarys.append(np.median(salary))
    
    print(salarys)
    df_text_salary = pd.DataFrame(zip(ad_list,salarys),columns=['ad_raw','salary'])
    print(df_text_salary)
    
    df_text_salary.to_csv(r'./raw_data.csv',index=False)
    
    nlp = spacy.load('en_core_web_sm')
    
    df_ad_vect_salary = df_text_salary.ad_raw.apply(nlp).apply(lambda x: pd.Series(x.vector))
    
    
    
    
    
    
    
    

In [ ]:
df_ad_vect_salary

In [ ]:
def list_mean(x):
    if x:
        np.mean(np.array(x))
        
def conditional_regex(input_string):
    import re
    day = re.compile('day')
    pa = re.compile('p\.a\.|p\s?a')
    hour = re.compile('p[\.\s]h.')

    input_string = input_string.lower().replace(',','')
    numbers = re.compile('\$(\d+)')

    if ('day' in input_string)|('daily' in input_string):
        return [float(number)*5*48 for number in numbers.findall(input_string) if float(number)>0]
    
    elif pa.search(input_string.lower()):
        return [float(number)*1000 for number in  numbers.findall(input_string) if float(number)>0]

    elif hour.search(input_string.lower()):
        return [float(number)*24*5*48 for number in  numbers.findall(input_string) if float(number)>0]
    elif numbers.search(input_string.lower()):
        return [float(number) for number in  numbers.findall(input_string) if float(number)>0]

def get_valid_salary(input_string):
    
    numbers = re.compile('(\d+)')
    
    salary = [num for num in numbers.findall(input_string) if(float(num)>0)&(float(num)<999999)]

    return salary[0] if salary else None

def handle_list(x):
    if x:
        return np.mean(np.array(x))
    else:
        return x
        
import re
numbers = re.compile('\$(\d+)')
new_dfs = []
for df in dfs:
    new_dfs.append(df)
    
    if df.filter(regex='salary_data_text').shape[1]>0:
        new_dfs[-1]['salary'] = new_dfs[-1]['salary_data_text'].map(lambda x: handle_list(conditional_regex(x)) if isinstance(x,str) else x)
    if df.filter(regex='salary_range').shape[1]>0:
        new_dfs[-1]['salary'] = new_dfs[-1]['salary_range'].astype(str).map(get_valid_salary)
    if pd.api.types.is_object_dtype(df['salary']):
        new_dfs[-1]['salary'] = new_dfs[-1]['salary'].astype(str).map(get_valid_salary)